In [1]:
from logging import PlaceHolder
from jupyter_dash import JupyterDash
# import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

In [2]:
import plotly.express as px
import pandas as pd

In [3]:
df = px.data.gapminder()
df_2 =  px.data.stocks()

In [4]:
df.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4


In [5]:
df_2.head()

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708


In [6]:
app = JupyterDash()

In [7]:
app.layout = html.Div([
    
    html.H1('Interactive Plots', style={'textAlign':'center'}),
    dcc.Graph(id='myGraph', figure={}),
    dcc.Graph(id='Graph2', figure={}),
    dcc.Graph(id='Graph3', figure={}),
    
    dcc.Dropdown(
        id = 'demo-dropdown',
        options = [
            {'label':str(continent),
             'value': str(continent)} for continent in df['continent'].unique()
        ],
        value = None,
        multi = True,
        placeholder = 'choose a continent ...'
    ),
    
    dcc.Slider(
        id = 'my-slider',
        min = df['year'].min(),
        max = df['year'].max(),
        value = df['year'].min(),
        step = None,
        marks = {str(years) : str(years) for years in df['year'].unique()}
    )
])

In [8]:
@app.callback(
    Output('myGraph', 'figure'),
    Output('Graph2', 'figure'),
    Output('Graph3', 'figure'),
    
    Input('my-slider', 'value'),
    Input('demo-dropdown', 'value')
)
def update_graph(slidervalue, dropdownvalue):
    if dropdownvalue == None:
        filtered_df = df[df['year'] == slidervalue]
        fig1 = px.scatter(filtered_df, x='gdpPercap', y='lifeExp',
                       color='continent', size='pop',
                       size_max=40, log_x=True,
                       title="My Scatter",
                       hover_name='country',
                      animation_frame = 'year',
                         template='plotly_dark')
    
    else:
        filtered_df = df[(df['year']==slidervalue) & (df['continent'].isin(dropdownvalue))]
        fig1 = px.scatter(filtered_df, x='gdpPercap', y='lifeExp', 
                         color='continent', size='pop',
                        size_max=40, log_x=True,
                        title="My Scatter",
                        hover_name='country',
                        animation_frame = 'year',
                         template='plotly_dark')
    
    fig2 = px.line(df_2, x='date', y=['GOOG', 'FB', 'AAPL'],
                   template='plotly_dark', markers='date')
    fig3 = px.choropleth(df, color='lifeExp', locations='iso_alpha', 
            hover_name='country', animation_frame = 'year', height=600)
    return fig2, fig3, fig1

In [9]:
if __name__ == '__main__':
    app.run_server(port=5070)

Dash app running on http://127.0.0.1:5070/
